In [2]:
# !pip install tensorflow

In [3]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [4]:
csv_path = 'inp/IBIRITE (ROLA MOCA)_MG.csv'

In [5]:
data = pd.read_csv(csv_path, sep=';')
data

,DATE,Chuva,$Pressão_{Inst}$,$Pressão_{Max}$,$Pressão_{Min}$,$Temperatura_{Inst}$,$Orvalho_{Inst}$,$Temperatura_{Max}$,$Temperatura_{Min}$,$Orvalho_{Max}$,...,$Umidade_{Min}$,$Umidade_{Inst}$,$Vento_{Dir}$,$Vento_{Raj}$,$Vento_{Vel}$,GLOBAL,Latitude,Longitude,Altitude,Qo
0,2008-06-06,0.000000,886.500000,886.533333,886.300000,21.100000,10.500000,22.333333,20.900000,10.766667,...,48.333333,51.666667,56.333333,6.233333,3.133333,1.9223,-20.031457,-44.011249,1198.82,2.195897
1,2008-06-07,0.000000,887.037500,887.250000,886.854167,19.458333,10.083333,20.016667,18.770833,10.637500,...,54.000000,56.458333,76.458333,6.604167,3.537500,19.2513,-20.031457,-44.011249,1198.82,24.544495
2,2008-06-08,0.000000,886.783333,886.983333,886.583333,19.095833,10.504167,19.704167,18.458333,11.054167,...,55.458333,58.750000,79.791667,8.570833,4.383333,17.9175,-20.031457,-44.011249,1198.82,24.486118
3,2008-06-09,0.000000,886.341667,886.516667,886.204167,18.700000,12.312500,19.304167,18.079167,12.662500,...,64.541667,67.583333,125.875000,7.779167,4.033333,16.2483,-20.031457,-44.011249,1198.82,24.430901
4,2008-06-10,0.000000,885.012500,885.220833,884.820833,18.529167,11.920833,19.291667,17.795833,12.566667,...,64.916667,67.833333,139.625000,4.750000,2.329167,13.3562,-20.031457,-44.011249,1198.82,24.378872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4792,2021-12-27,0.000000,880.675000,880.900000,880.425000,22.075000,14.425000,22.783333,21.395833,15.054167,...,60.291667,63.416667,174.083333,6.562500,1.462500,26.3813,-20.031457,-44.011249,1198.82,42.703244
4793,2021-12-28,0.983333,881.758333,882.012500,881.445833,21.458333,15.970833,22.525000,20.708333,16.695833,...,68.333333,72.875000,176.750000,7.200000,1.875000,23.1625,-20.031457,-44.011249,1198.82,42.712820
4794,2021-12-29,0.150000,883.337500,883.616667,883.075000,19.704167,16.345833,20.341667,19.187500,16.850000,...,78.791667,82.000000,131.375000,5.400000,0.254167,18.4593,-20.031457,-44.011249,1198.82,42.718808
4795,2021-12-30,1.008333,881.166667,881.458333,880.958333,18.454167,16.762500,18.820833,18.204167,17.050000,...,88.375000,89.875000,98.000000,5.812500,1.733333,5.3534,-20.031457,-44.011249,1198.82,42.721202


In [6]:
def remove_timezone(dt):
    # HERE `dt` is a python datetime
    # object that used .replace() method
    return dt.replace(tzinfo=None)

In [7]:
# A PARTIR DE 2019 O FORMATO DE DATA DAS TABELAS FOI MODIFICADO, ENTÃO TIVE QUE O TIMEZONE SOMENTE DESTAS ENTRADAS
# data['DATE'].loc[data['DATE'].apply(lambda t: t.tzinfo is not None)]=data['DATE'].loc[data['DATE'].apply(lambda t: t.tzinfo is not None)].apply(remove_timezone)




In [8]:
data.rename(columns={"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "Chuva", 
                     "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "$Pressão_{Inst}$",
                     "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)":"$Pressão_{Max}$",
                     "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)":"$Pressão_{Min}$",
                     "RADIACAO GLOBAL (KJ/m²)":"H",
                     "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)":"$Temperatura_{Inst}$",
                     "TEMPERATURA DO PONTO DE ORVALHO (°C)":"$Orvalho_{Inst}$",
                     "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)":"$Temperatura_{Max}$",
                     "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)":"$Temperatura_{Min}$",
                     "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)":"$Orvalho_{Max}$",
                     "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)":"$Orvalho_{Min}$",
                     "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)":"$Umidade_{Max}$",
                     "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)":"$Umidade_{Min}$",
                     "UMIDADE RELATIVA DO AR, HORARIA (%)":"$Umidade_{Inst}$",
                     "VENTO, DIREÇÃO HORARIA (gr) (° (gr))":"$Vento_{Dir}$",
                     "VENTO, RAJADA MAXIMA (m/s)":"$Vento_{Raj}$",
                     "VENTO, VELOCIDADE HORARIA (m/s)":"$Vento_{Vel}$",
                    "RADIACAO GLOBAL (Kj/m²)":"GLOBAL"},inplace=True)
data

,DATE,Chuva,$Pressão_{Inst}$,$Pressão_{Max}$,$Pressão_{Min}$,$Temperatura_{Inst}$,$Orvalho_{Inst}$,$Temperatura_{Max}$,$Temperatura_{Min}$,$Orvalho_{Max}$,...,$Umidade_{Min}$,$Umidade_{Inst}$,$Vento_{Dir}$,$Vento_{Raj}$,$Vento_{Vel}$,GLOBAL,Latitude,Longitude,Altitude,Qo
0,2008-06-06,0.000000,886.500000,886.533333,886.300000,21.100000,10.500000,22.333333,20.900000,10.766667,...,48.333333,51.666667,56.333333,6.233333,3.133333,1.9223,-20.031457,-44.011249,1198.82,2.195897
1,2008-06-07,0.000000,887.037500,887.250000,886.854167,19.458333,10.083333,20.016667,18.770833,10.637500,...,54.000000,56.458333,76.458333,6.604167,3.537500,19.2513,-20.031457,-44.011249,1198.82,24.544495
2,2008-06-08,0.000000,886.783333,886.983333,886.583333,19.095833,10.504167,19.704167,18.458333,11.054167,...,55.458333,58.750000,79.791667,8.570833,4.383333,17.9175,-20.031457,-44.011249,1198.82,24.486118
3,2008-06-09,0.000000,886.341667,886.516667,886.204167,18.700000,12.312500,19.304167,18.079167,12.662500,...,64.541667,67.583333,125.875000,7.779167,4.033333,16.2483,-20.031457,-44.011249,1198.82,24.430901
4,2008-06-10,0.000000,885.012500,885.220833,884.820833,18.529167,11.920833,19.291667,17.795833,12.566667,...,64.916667,67.833333,139.625000,4.750000,2.329167,13.3562,-20.031457,-44.011249,1198.82,24.378872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4792,2021-12-27,0.000000,880.675000,880.900000,880.425000,22.075000,14.425000,22.783333,21.395833,15.054167,...,60.291667,63.416667,174.083333,6.562500,1.462500,26.3813,-20.031457,-44.011249,1198.82,42.703244
4793,2021-12-28,0.983333,881.758333,882.012500,881.445833,21.458333,15.970833,22.525000,20.708333,16.695833,...,68.333333,72.875000,176.750000,7.200000,1.875000,23.1625,-20.031457,-44.011249,1198.82,42.712820
4794,2021-12-29,0.150000,883.337500,883.616667,883.075000,19.704167,16.345833,20.341667,19.187500,16.850000,...,78.791667,82.000000,131.375000,5.400000,0.254167,18.4593,-20.031457,-44.011249,1198.82,42.718808
4795,2021-12-30,1.008333,881.166667,881.458333,880.958333,18.454167,16.762500,18.820833,18.204167,17.050000,...,88.375000,89.875000,98.000000,5.812500,1.733333,5.3534,-20.031457,-44.011249,1198.82,42.721202


In [9]:
data.set_index('DATE',inplace=True)

In [10]:
data.dropna(axis=0, how='all', subset=None, inplace=True)
data.dropna(axis=1, how='all', subset=None, inplace=True)

In [11]:
#DEPENDENDO DO ANO E DA ESTAÇÃO OS VALORES DE RADIAÇÃO ESTÃO EM COLUNAS DIFERENTES
if 'GLOBAL' not in data.columns:
    data.rename(columns={"H": "GLOBAL"}, inplace=True)
if 'H' in data.columns:
    data['GLOBAL'].fillna(data['H'], inplace=True)
    data.drop(['H'], axis=1, inplace=True)

In [12]:
    #GARANTINDO QUE TODOS OS DADOS ESTÃO SENDO TRATADOS COMO FLOAT
    data = data.replace(',','.', regex=True)
    data['Chuva']=data['Chuva'].astype(float)
    data['$Pressão_{Inst}$']=data['$Pressão_{Inst}$'].astype(float)
    data['$Pressão_{Max}$']=data['$Pressão_{Max}$'].astype(float)
    data['$Pressão_{Min}$']=data['$Pressão_{Min}$'].astype(float)
    data['$Temperatura_{Inst}$']=data['$Temperatura_{Inst}$'].astype(float)
    data['$Temperatura_{Max}$']=data['$Temperatura_{Max}$'].astype(float)
    data['$Temperatura_{Min}$']=data['$Temperatura_{Min}$'].astype(float)
    data['$Orvalho_{Inst}$']=data['$Orvalho_{Inst}$'].astype(float)
    data['$Orvalho_{Min}$']=data['$Orvalho_{Min}$'].astype(float)
    data['$Orvalho_{Max}$']=data['$Orvalho_{Max}$'].astype(float)
    data['$Vento_{Raj}$']=data['$Vento_{Raj}$'].astype(float)
    data['$Vento_{Vel}$']=data['$Vento_{Vel}$'].astype(float)
    data['GLOBAL']=data['GLOBAL'].astype(float)

In [13]:
# data = data[['GLOBAL', 'Chuva', '$Pressão_{Inst}$']]


In [14]:
#DELETANDO ALGUMA LEITURA ONDE O MÁXIMO É MENOR QUE O MÍNIMO
delIndex=data.loc[(data['$Pressão_{Max}$']<data['$Pressão_{Min}$'])|(data['$Temperatura_{Max}$']<data['$Temperatura_{Min}$'])|(data['$Orvalho_{Max}$']<data['$Orvalho_{Min}$'])|(data['$Umidade_{Max}$']<data['$Umidade_{Min}$'])].index
if len(delIndex)>0:
    data.drop(delIndex,axis=0, inplace=True)

In [15]:
#DELETANDO ALGUMA LEITURA ONDE O MÁXIMO É MENOR QUE O INSTANTÂNEO
delIndex=data.loc[(data['$Pressão_{Max}$']<data['$Pressão_{Inst}$'])|(data['$Temperatura_{Max}$']<data['$Temperatura_{Inst}$'])|(data['$Orvalho_{Max}$']<data['$Orvalho_{Inst}$'])|(data['$Umidade_{Max}$']<data['$Umidade_{Inst}$'])].index
if len(delIndex)>0:
    data.drop(delIndex,axis=0, inplace=True)

In [16]:
#DELETANDO ALGUMA LEITURA ONDE O INSTANTÂNEO É MENOR QUE O MÍNIMO
delIndex=data.loc[(data['$Pressão_{Inst}$']<data['$Pressão_{Min}$'])|(data['$Temperatura_{Inst}$']<data['$Temperatura_{Min}$'])|(data['$Orvalho_{Inst}$']<data['$Orvalho_{Min}$'])|(data['$Umidade_{Inst}$']<data['$Umidade_{Min}$'])].index
if len(delIndex)>0:
    data.drop(delIndex,axis=0, inplace=True)

In [17]:
# #MODIFICANDO A UNIDADE DA RADIAÇÃO DE (Kj/m²) PARA (Mj/m²)
data['GLOBAL'] = data['GLOBAL'].mul(0.001)

In [18]:
#DELETANDO ALGUMA LINHA QUE AINDA TENHA NAN
data.dropna(inplace=True)

In [19]:
# EXTRAINDO OS NOMES DAS COLUNAS 
target_names=['GLOBAL']
variable_names=data.columns.values
variable_names = np.delete(variable_names, np.where(variable_names==target_names))

In [20]:
date_range = data.index  

var_to_plot=data.columns
df=data[var_to_plot]

n=int(df.shape[0]*0.7)
df.index=range(df.shape[0])
id0=df.index <= n
id1=df.index >  n 
print(data.index)

Index(['2008-06-06', '2008-06-07', '2008-06-08', '2008-06-09', '2008-06-10',
       '2008-06-11', '2008-06-12', '2008-06-13', '2008-06-14', '2008-06-15',
       ...
       '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26',
       '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31'],
      dtype='object', name='DATE', length=4797)


In [21]:
X=data[variable_names]
y=data[target_names]

In [22]:
df=X.copy()

In [23]:
# !pip install pylab
# import pylab as pl

In [24]:
df[target_names]=y.values
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
# f, ax = pl.subplots(figsize=(11, 9))
# cmap = sns.diverging_palette(230, 20, as_cmap=True)
# cmap =  cmap="YlGnBu"
# sns.heatmap(corr, mask=mask, cmap=cmap, #vmax=.3, center=0, 
#            annot=True, #fmt="d")
#            square=True, linewidths=.5, #cbar_kws={"shrink": .5},
#        )
# pl.show()

In [27]:
def df_to_X_y(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np) - window_size):
    row = [[a] for a in df_as_np[i:i + window_size]]
    X.append(row)
    label = df_as_np[i + window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [28]:
WINDOW_SIZE = 3
X1, y1 = df_to_X_y(data, WINDOW_SIZE)
X1.shape, y1.shape
X1, y1

(array([[[[ 0.00000000e+00,  8.86500000e+02,  8.86533333e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.19589739e+00]],
 
         [[ 0.00000000e+00,  8.87037500e+02,  8.87250000e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.45444955e+01]],
 
         [[ 0.00000000e+00,  8.86783333e+02,  8.86983333e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.44861184e+01]]],
 
 
        [[[ 0.00000000e+00,  8.87037500e+02,  8.87250000e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.45444955e+01]],
 
         [[ 0.00000000e+00,  8.86783333e+02,  8.86983333e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.44861184e+01]],
 
         [[ 0.00000000e+00,  8.86341667e+02,  8.86516667e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.44309008e+01]]],
 
 
        [[[ 0.00000000e+00,  8.86783333e+02,  8.86983333e+02, ...,
           -4.40112490e+01,  1.19882000e+03,  2.44861184e+01]],
 
         [[ 0.00000000e+00,  8.86341667e+02,  8.86516667e+02, .

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X1, y1, test_size=0.2, shuffle=True)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True)



In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17,425
Trainable params: 17,425
Non-trainable params: 0
_________________________________________________________________


In [31]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [32]:
history1 = model1.fit(train_X, train_y, validation_data=(X_val, Y_val), epochs=10, callbacks=[cp1])

NameError: name 'train_X' is not defined